## Задача 3
### Часть 2

Нужно, используя Spark: 
1. найти явку (%) по всем регионам, результат отсортировать по убыванию 
2. выбрать любимого кандидата и найти тот избиратльный участок, на котором он получил наибольший результат (учитывать участки на которых проголосовало больше 300 человек) 
3. найти регион, где разница между ТИК с наибольшей явкой и наименьшей максимальна 
4. посчитать дисперсию по явке для каждого региона (учитывать УИК) 
5. для каждого кандидата посчитать таблицу: результат (%, округленный до целого) 
6. количество УИК, на которых кандидат получил данный результат

Результаты принимаются в виде Jupyter Notebook, Spark Notebook или исходных файлов на Scala.

In [150]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from functools import reduce
from operator import add
from pyspark.sql.functions import col
from pyspark.sql.functions import lit

### Чтение данных

In [2]:
data_file = 'data.csv'

In [3]:
spark = SparkSession.builder.appName("PySpark").getOrCreate()

In [4]:
df = spark.read.csv(data_file, inferSchema=True, header=True).toDF( \
    'Регион','ТИК','УИК', 'Избиратели', 'Бюллетени', \
    'Бюллетени досрочно', 'Бюллетени внутри', 'Бюллетени снаружи', 'Бюллетени погашенные', \
    'В переносных ящиках', 'В стационарных ящиках', \
    'Бюллетени недействительные', 'Бюллетени действительные', 'Бюллетени утраченные', 'Бюллетени неучтенные', \
    'Бабурин', 'Грудинин', 'Жириновский', 'Путин', \
    'Собчак', 'Сурайкин', 'Титов', 'Явлинский')

### 1. Явка (%) по всем регионам, отсортированная по убыванию

In [18]:
df_presence = df.groupBy('Регион') \
    .agg(F.sum('Избиратели').alias('Кол-во избирателей'), \
        F.sum('Бюллетени досрочно').alias('Кол-во бюллетеней досрочно'), \
        F.sum('Бюллетени внутри').alias('Кол-во бюллетеней внутри'), \
        F.sum('Бюллетени снаружи').alias('Кол-во бюллетеней снаружи')) \
    .sort('Кол-во избирателей', ascending=False)

In [19]:
df_presence.show(10)

+--------------------+------------------+--------------------------+------------------------+-------------------------+
|              Регион|Кол-во избирателей|Кол-во бюллетеней досрочно|Кол-во бюллетеней внутри|Кол-во бюллетеней снаружи|
+--------------------+------------------+--------------------------+------------------------+-------------------------+
|        город Москва|           7543682|                         0|                 4348576|                   172779|
|  Московская область|           5829578|                         0|                 3540550|                   167006|
|  Краснодарский край|           4049575|                       229|                 2825178|                   327900|
|город Санкт-Петер...|           3632398|                      1795|                 2248088|                    70251|
|Свердловская область|           3345221|                      3210|                 2003456|                    79001|
|  Ростовская область|           3210796

In [20]:
all_votes = ['Кол-во бюллетеней досрочно', 'Кол-во бюллетеней внутри', 'Кол-во бюллетеней снаружи']

df_presence = df_presence \
    .withColumn('Кол-во голосов', reduce(add, [col(votes) for votes in all_votes])) \
    .select('Регион', 'Кол-во избирателей', 'Кол-во голосов') \
    .sort('Кол-во избирателей', ascending=False)

In [21]:
df_presence.show(10)

+--------------------+------------------+--------------+
|              Регион|Кол-во избирателей|Кол-во голосов|
+--------------------+------------------+--------------+
|        город Москва|           7543682|       4521355|
|  Московская область|           5829578|       3707556|
|  Краснодарский край|           4049575|       3153307|
|город Санкт-Петер...|           3632398|       2320134|
|Свердловская область|           3345221|       2085667|
|  Ростовская область|           3210796|       2079778|
|Республика Башкор...|           3045698|       2297834|
|Республика Татарс...|           2919482|       2260309|
| Челябинская область|           2632596|       1748424|
|Нижегородская обл...|           2618865|       1728008|
+--------------------+------------------+--------------+
only showing top 10 rows



In [22]:
df_presence = df_presence \
    .withColumn('Явка (%)', 100 * col('Кол-во голосов') / col('Кол-во избирателей')) \
    .select('Регион', 'Явка (%)') \
    .sort('Явка (%)', ascending=False)

In [23]:
df_presence.show(10)

+--------------------+-----------------+
|              Регион|         Явка (%)|
+--------------------+-----------------+
|Территория за пре...|98.06986984380843|
|     Республика Тыва|93.66369316170004|
|Ямало-Ненецкий ав...|91.90125747324196|
|Кабардино-Балкарс...| 91.8015786356213|
|Чеченская Республика|91.54001705101919|
|Республика Северн...|89.98810071492768|
| Республика Дагестан|87.47795414462081|
|Карачаево-Черкесс...|87.40544668203033|
| Кемеровская область|83.22733516003962|
|Чукотский автоном...| 82.2813869592439|
+--------------------+-----------------+
only showing top 10 rows



### 2. Избирательный участок, на котором выбранный кандидат получил наибольший результат, учитывая только те участки, на которых более 300 проголосовавших

In [94]:
candidates = ['Бабурин', 'Грудинин', 'Жириновский', 'Путин', 'Собчак', 'Сурайкин', 'Титов', 'Явлинский']

In [95]:
def get_max_for_candidate(candidate):
    n = 300
    result = df.filter(col(candidate) > n) \
        .select('Регион', 'ТИК', 'УИК', candidate) \
        .groupBy('Регион', 'ТИК', 'УИК')\
        .agg(F.max(candidate).alias('Кол-во голосов')) \
        .sort('Кол-во голосов',  ascending=False)
    return result.show(1)

In [100]:
get_max_for_candidate(candidates[2])

+-----------------+-------------------+---------+--------------+
|           Регион|                ТИК|      УИК|Кол-во голосов|
+-----------------+-------------------+---------+--------------+
|Тюменская область|Тюмень, Калининская|УИК №2081|           421|
+-----------------+-------------------+---------+--------------+
only showing top 1 row



### 3. Регион, где разница между ТИК с наибольшей явкой и наименьшей максимальна 

In [132]:
df_tiks = df.groupBy('Регион', 'ТИК') \
    .agg(F.sum('Избиратели').alias('Кол-во избирателей'), \
         F.sum('Бюллетени досрочно').alias('Кол-во бюллетеней досрочно'), \
         F.sum('Бюллетени внутри').alias('Кол-во бюллетеней внутри'), \
         F.sum('Бюллетени снаружи').alias('Кол-во бюллетеней снаружи')) \
    .withColumn('Кол-во голосов', reduce(add, [col(votes) for votes in all_votes])) \
    .withColumn('Явка (%)', 100 * col('Кол-во голосов') / col('Кол-во избирателей')) \
    .select('Регион', 'ТИК', 'Явка (%)') \
    .sort('Регион')

In [133]:
df_tiks.show(10)

+--------------+--------------------+-----------------+
|        Регион|                 ТИК|         Явка (%)|
+--------------+--------------------+-----------------+
|Алтайский край|        Кытмановская|73.75977819586097|
|Алтайский край|        Ребрихинская|66.21323934354004|
|Алтайский край|     Усть-Калманская|63.29329585143539|
|Алтайский край|          Солтонская|69.21678533555522|
|Алтайский край|     Быстроистокская|65.77316810344827|
|Алтайский край|        Михайловская|61.26126126126126|
|Алтайский край|          Курьинская|67.64493625210488|
|Алтайский край|         Шипуновская|65.33391153512576|
|Алтайский край|       Третьяковская|67.97066014669926|
|Алтайский край|Новоалтайская гор...|64.66214928166646|
+--------------+--------------------+-----------------+
only showing top 10 rows



In [134]:
df_tiks = df_tiks.groupBy('Регион') \
    .agg((F.max('Явка (%)') - F.min('Явка (%)')).alias('Разница')) \
    .select('Регион', 'Разница') \
    .sort('Разница', ascending=False)

In [136]:
df_tiks.show(1)

+--------------------+----------------+
|              Регион|         Разница|
+--------------------+----------------+
|Архангельская обл...|49.8591648406849|
+--------------------+----------------+
only showing top 1 row



### 4. Дисперсия по явке для каждого региона (учитывая УИК) 

In [141]:
df_disp = df.groupBy('Регион', 'ТИК', 'УИК') \
    .agg(F.sum('Избиратели').alias('Кол-во избирателей'), \
         F.sum('Бюллетени досрочно').alias('Кол-во бюллетеней досрочно'), \
         F.sum('Бюллетени внутри').alias('Кол-во бюллетеней внутри'), \
         F.sum('Бюллетени снаружи').alias('Кол-во бюллетеней снаружи')) \
    .withColumn('Кол-во голосов', reduce(add, [col(votes) for votes in all_votes])) \
    .withColumn('Явка (%)', 100 * col('Кол-во голосов') / col('Кол-во избирателей')) \
    .select('Регион', 'Явка (%)') \
    .groupBy('Регион') \
    .agg(F.stddev('Явка (%)').alias('Дисперсия')) \
    .sort('Дисперсия', ascending=False)

In [142]:
df_disp.show(10)

+--------------------+------------------+
|              Регион|         Дисперсия|
+--------------------+------------------+
| Сахалинская область|  20.1817713258144|
|     Камчатский край|16.861471602822412|
|  Мурманская область|16.734301006169574|
|     Приморский край|16.641672216989203|
|Республика Адыгея...|16.330124345063627|
|   Самарская область| 15.60463973003109|
| Саратовская область|15.419177745102223|
| Республика Калмыкия| 15.34992596054361|
| Воронежская область|14.969001494440173|
| Магаданская область|14.380287651040703|
+--------------------+------------------+
only showing top 10 rows



### 5. Таблица для каждого кандидата: результат (%, округленный до целого) 

In [264]:
df_region = df.groupBy('Регион') \
    .agg(F.sum(col(candidates[0])).alias(candidates[0]), \
         F.sum(col(candidates[1])).alias(candidates[1]), \
         F.sum(col(candidates[2])).alias(candidates[2]), \
         F.sum(col(candidates[3])).alias(candidates[3]), \
         F.sum(col(candidates[4])).alias(candidates[4]), \
         F.sum(col(candidates[5])).alias(candidates[5]), \
         F.sum(col(candidates[6])).alias(candidates[6]), \
         F.sum(col(candidates[7])).alias(candidates[7])) \
    .sort('Регион', ascending=True)

In [265]:
df_region.show(10)

+--------------------+-------+--------+-----------+------+------+--------+-----+---------+
|              Регион|Бабурин|Грудинин|Жириновский| Путин|Собчак|Сурайкин|Титов|Явлинский|
+--------------------+-------+--------+-----------+------+------+--------+-----+---------+
|      Алтайский край|   7581|  281978|      84785|770278| 11788|    7855| 5532|     7259|
|    Амурская область|   2358|   73485|      37909|264493|  4428|    2466| 2080|     1951|
|Архангельская обл...|   4448|   51868|      46925|407190| 10588|    3842| 4982|     6239|
|Астраханская область|   2185|   64047|      19339|342195|  5060|    2823| 2233|     2504|
|Белгородская область|   5218|   93102|      49685|711392|  8474|    6534| 4835|     4445|
|    Брянская область|   4472|   68375|      43940|636087|  7463|    4265| 4175|     3524|
|Владимирская область|   5440|   93649|      58822|546042| 10777|    5075| 6098|     6147|
|Волгоградская обл...|   8040|  140708|      69909|929541| 14403|    8116| 6851|    10242|

In [266]:
df_russia = df_region \
    .withColumn('Страна', lit('Россия')) \
    .groupBy('Страна') \
    .agg(F.sum(col(candidates[0])).alias(candidates[0]), \
         F.sum(col(candidates[1])).alias(candidates[1]), \
         F.sum(col(candidates[2])).alias(candidates[2]), \
         F.sum(col(candidates[3])).alias(candidates[3]), \
         F.sum(col(candidates[4])).alias(candidates[4]), \
         F.sum(col(candidates[5])).alias(candidates[5]), \
         F.sum(col(candidates[6])).alias(candidates[6]), \
         F.sum(col(candidates[7])).alias(candidates[7])) 

In [267]:
df_russia.show()

+------+-------+--------+-----------+--------+-------+--------+------+---------+
|Страна|Бабурин|Грудинин|Жириновский|   Путин| Собчак|Сурайкин| Титов|Явлинский|
+------+-------+--------+-----------+--------+-------+--------+------+---------+
|Россия| 475599| 8608210|    4124987|56048674|1225330|  496001|552007|   762304|
+------+-------+--------+-----------+--------+-------+--------+------+---------+



In [268]:
df_total_amount = df_russia \
    .withColumn('Страна', lit('Россия')) \
    .withColumn('Общее кол-во голосов', reduce(add, [col(candidate) for candidate in candidates])) \
    .select('Страна', 'Общее кол-во голосов')

In [269]:
df_total_amount.show()

+------+--------------------+
|Страна|Общее кол-во голосов|
+------+--------------------+
|Россия|            72293112|
+------+--------------------+



In [270]:
total_amount = df_total_amount.collect()[0][1]
total_amount

72293112

In [271]:
df_russia_percent = df_region \
    .withColumn('Страна', lit('Россия')) \
    .groupBy('Страна') \
    .agg((F.round(100 * F.sum(col(candidates[0])) / total_amount)).alias(candidates[0] + ' (%)'), \
         (F.round(100 * F.sum(col(candidates[1])) / total_amount)).alias(candidates[1] + ' (%)'), \
         (F.round(100 * F.sum(col(candidates[2])) / total_amount)).alias(candidates[2] + ' (%)'), \
         (F.round(100 * F.sum(col(candidates[3])) / total_amount)).alias(candidates[3] + ' (%)'), \
         (F.round(100 * F.sum(col(candidates[4])) / total_amount)).alias(candidates[4] + ' (%)'), \
         (F.round(100 * F.sum(col(candidates[5])) / total_amount)).alias(candidates[5] + ' (%)'), \
         (F.round(100 * F.sum(col(candidates[6])) / total_amount)).alias(candidates[6] + ' (%)'), \
         (F.round(100 * F.sum(col(candidates[7])) / total_amount)).alias(candidates[7] + ' (%)')) 

In [272]:
df_russia_percent.show()

+------+-----------+------------+---------------+---------+----------+------------+---------+-------------+
|Страна|Бабурин (%)|Грудинин (%)|Жириновский (%)|Путин (%)|Собчак (%)|Сурайкин (%)|Титов (%)|Явлинский (%)|
+------+-----------+------------+---------------+---------+----------+------------+---------+-------------+
|Россия|        1.0|        12.0|            6.0|     78.0|       2.0|         1.0|      1.0|          1.0|
+------+-----------+------------+---------------+---------+----------+------------+---------+-------------+



In [279]:
total_amounts = [df_russia_percent.collect()[0][cand_ind + 1] for cand_ind in range(len(candidates))]
total_amounts

[1.0, 12.0, 6.0, 78.0, 2.0, 1.0, 1.0, 1.0]

### 6. Количество УИК, на которых кандидат получил данный результат

In [273]:
df_uik = df.groupBy('Регион', 'УИК') \
    .agg(F.sum(col(candidates[0])).alias(candidates[0]), \
         F.sum(col(candidates[1])).alias(candidates[1]), \
         F.sum(col(candidates[2])).alias(candidates[2]), \
         F.sum(col(candidates[3])).alias(candidates[3]), \
         F.sum(col(candidates[4])).alias(candidates[4]), \
         F.sum(col(candidates[5])).alias(candidates[5]), \
         F.sum(col(candidates[6])).alias(candidates[6]), \
         F.sum(col(candidates[7])).alias(candidates[7])) \
    .withColumn('Общее кол-во голосов', reduce(add, [col(candidate) for candidate in candidates])) \
    .sort('Регион', ascending=True)

In [274]:
df_uik.show(10)

+--------------+---------+-------+--------+-----------+-----+------+--------+-----+---------+--------------------+
|        Регион|      УИК|Бабурин|Грудинин|Жириновский|Путин|Собчак|Сурайкин|Титов|Явлинский|Общее кол-во голосов|
+--------------+---------+-------+--------+-----------+-----+------+--------+-----+---------+--------------------+
|Алтайский край|УИК №1399|      5|     159|         47|  301|     5|       7|    1|        0|                 525|
|Алтайский край| УИК №379|     13|     394|         84|  836|    16|      13|   11|       15|                1382|
|Алтайский край| УИК №925|      1|       9|          2|  102|     1|       0|    0|        2|                 117|
|Алтайский край|УИК №1314|      1|      16|          8|   72|     0|       1|    0|        0|                  98|
|Алтайский край|УИК №1518|      2|     119|         31|  334|     2|       0|    0|        2|                 490|
|Алтайский край| УИК №468|      8|     402|         78|  777|     5|       8|   

In [291]:
df_uik_percent = df_uik \
    .withColumn(candidates[0] + ' (%)', F.round(100 * col(candidates[0]) / col('Общее кол-во голосов'))) \
    .withColumn(candidates[1] + ' (%)', F.round(100 * col(candidates[1]) / col('Общее кол-во голосов'))) \
    .withColumn(candidates[2] + ' (%)', F.round(100 * col(candidates[2]) / col('Общее кол-во голосов'))) \
    .withColumn(candidates[3] + ' (%)', F.round(100 * col(candidates[3]) / col('Общее кол-во голосов'))) \
    .withColumn(candidates[4] + ' (%)', F.round(100 * col(candidates[4]) / col('Общее кол-во голосов'))) \
    .withColumn(candidates[5] + ' (%)', F.round(100 * col(candidates[5]) / col('Общее кол-во голосов'))) \
    .withColumn(candidates[6] + ' (%)', F.round(100 * col(candidates[6]) / col('Общее кол-во голосов'))) \
    .withColumn(candidates[7] + ' (%)', F.round(100 * col(candidates[7]) / col('Общее кол-во голосов'))) \
    .select('Регион', 'УИК', candidates[0] + ' (%)', candidates[1] + ' (%)', candidates[2] + ' (%)', \
           candidates[3] + ' (%)', candidates[4] + ' (%)', candidates[5] + ' (%)', \
           candidates[6] + ' (%)', candidates[7] + ' (%)') \
    .sort('Регион', 'УИК', ascending=True)

In [292]:
df_uik_percent.show(10)

+--------------+---------+-----------+------------+---------------+---------+----------+------------+---------+-------------+
|        Регион|      УИК|Бабурин (%)|Грудинин (%)|Жириновский (%)|Путин (%)|Собчак (%)|Сурайкин (%)|Титов (%)|Явлинский (%)|
+--------------+---------+-----------+------------+---------------+---------+----------+------------+---------+-------------+
|Алтайский край|   УИК №1|        1.0|        23.0|            9.0|     65.0|       1.0|         1.0|      0.0|          1.0|
|Алтайский край|  УИК №10|        0.0|        23.0|           11.0|     63.0|       1.0|         1.0|      0.0|          1.0|
|Алтайский край| УИК №100|        1.0|        28.0|            5.0|     62.0|       3.0|         0.0|      1.0|          1.0|
|Алтайский край|УИК №1000|        1.0|        14.0|            6.0|     78.0|       0.0|         1.0|      0.0|          0.0|
|Алтайский край|УИК №1001|        0.0|         9.0|           15.0|     76.0|       0.0|         0.0|      0.0|       

In [322]:
def get_matches_for_candidate(cand_ind):
    df_mathes = df_uik_percent.filter(col(candidates[cand_ind] + ' (%)') == total_amounts[cand_ind]) \
        .select('Регион', 'УИК', candidates[cand_ind] + ' (%)') \
        .sort('Регион', 'УИК', ascending=True)
    return df_mathes 

In [323]:
def get_UIK_count(cand_ind):
    df_mathes_cand = get_matches_for_candidate(cand_ind)
    return df_mathes_cand.count()

In [324]:
def get_UIK_counts():
    UIK_counts = []
    for cand_ind in range(len(candidates)):
        UIK_count = get_UIK_count(cand_ind)
        UIK_counts.append(UIK_count)
    return UIK_counts

In [327]:
ar_UIK_counts = get_UIK_counts()

In [328]:
ar_UIK_counts

[47866, 7717, 13479, 4735, 18518, 48795, 39817, 30656]

In [342]:
result_columns = ['Название']
for name in candidates:
    result_columns.append(name)

In [345]:
result_data = []
amount_data = ['Результат (%)']
for amount in total_amounts:
    amount_data.append(int(amount))
result_data.append(amount_data)
counts_data = ['Кол-во УИК']
for count in ar_UIK_counts:
    counts_data.append(count)
result_data.append(counts_data)

In [347]:
df_result = spark.createDataFrame(result_data, result_columns)

In [348]:
df_result.show()

+-------------+-------+--------+-----------+-----+------+--------+-----+---------+
|     Название|Бабурин|Грудинин|Жириновский|Путин|Собчак|Сурайкин|Титов|Явлинский|
+-------------+-------+--------+-----------+-----+------+--------+-----+---------+
|Результат (%)|      1|      12|          6|   78|     2|       1|    1|        1|
|   Кол-во УИК|  47866|    7717|      13479| 4735| 18518|   48795|39817|    30656|
+-------------+-------+--------+-----------+-----+------+--------+-----+---------+

